In [2]:
import random
from copy import deepcopy
from math import log10

import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
from matplotlib.axes import Axes
from IPython.display import display
import requests
import os 
from dotenv import load_dotenv

In [3]:
# Load environment variables 
load_dotenv('simulation.env')

# Access environment variables 
subgraph_url = os.environ['SUBGRAPH_URL']
api_host = os.environ['API_HOST']
api_key = os.environ['API_KEY']

if 'SUBGRAPH_URL' in os.environ and 'API_HOST' in os.environ and 'API_KEY' in os.environ:
    print(True)
else:
    print(False)

True


### Get safe balances from subgraph and channel balances from the topology api endpoint 

In [11]:
def get_subgraph_data():
    """
    This function retrieves safe_address-node_address-balance links from the
    specified subgraph using pagination.
    """

    query = """
            query SafeNodeBalance($first: Int, $skip: Int) {
                safes(first: $first, skip: $skip) {
                    registeredNodesInNetworkRegistry {
                    node {
                        id
                    }
                    safe {
                        id
                        balance {
                        wxHoprBalance
                        }
                    }
                    }
                }
            }
        """

    data = {
        "query": query,
        "variables": {
            "first": 1000,
            "skip": 0,
        },
    }
    subgraph_dict = {}
    more_content_available = True
    pagination_skip_size = 1000

    while more_content_available:
        try:
            response = requests.post(subgraph_url, json=data)

            if response.status_code != 200:
                print(f"Received status code {response.status_code} when querying The Graph API")
                break

            json_data = response.json()

        except requests.exceptions.RequestException:
            print("An error occurred while sending the request to subgraph endpoint")
            return {}
        except ValueError:
            print("An error occurred while parsing the response as JSON from subgraph endpoint")
            return {}
        except Exception:
            print("An unexpected error occurred")
            return {}

        safes = json_data["data"]["safes"]
        for safe in safes:
            for node in safe["registeredNodesInNetworkRegistry"]:
                node_address = node["node"]["id"]
                wxHoprBalance = node["safe"]["balance"]["wxHoprBalance"]
                safe_address = node["safe"]["id"]
                subgraph_dict[node_address] = {
                    "safe_address": safe_address,
                    "wxHOPR_balance": wxHoprBalance,
                }

        # Increment skip for the next iteration
        data["variables"]["skip"] += pagination_skip_size
        more_content_available = len(safes) == pagination_skip_size

    return subgraph_dict

def get_unique_nodeAddress_peerId_aggbalance_links(api_url, api_key):
    """
    Returns a dict containing all unique source_peerId-source_address links.
    """
    channel_url = "http://{}:3001/api/v3/channels/?includingClosed=false&fullTopology=true".format(api_host)
    headers = {'X-Auth-Token': api_key}
    response = requests.request("GET", channel_url, headers=headers)

    if response.status_code != 200:
        print("Could not fetch channel information. Status code: {}".format(response.status_code))
        return {}
    
    response = response.json()

    if 'all' not in response:
            print("Response does not contain `all`")
            return {}

    peerid_address_aggbalance_links = {}
    for item in response["all"]:
        if "sourcePeerId" not in item or "sourceAddress" not in item:
            print("Response does not contain `source_peerid` or `source_address`")
            continue

        if "status" not in item:
            print("Response does not contain `status`")
            continue

        source_peer_id = item["sourcePeerId"]
        source_address = item["sourceAddress"]
        balance = int(item["balance"]) / 1e18

        if item["status"] != "Open":
            # Other Statuses: "Waiting for commitment", "Closed", "Pending to close"
            # Ensures that nodes must have at least 1 open channel in to receive ct
            continue

        if source_peer_id not in peerid_address_aggbalance_links:
            peerid_address_aggbalance_links[source_peer_id] = {
                "source_node_address": source_address,
                "channels_balance": balance,
            }

        else:
            peerid_address_aggbalance_links[source_peer_id][
                "channels_balance"
            ] += balance

    return peerid_address_aggbalance_links

In [15]:
topology_data = get_unique_nodeAddress_peerId_aggbalance_links(api_host, api_key)

print(len(topology_data))

print_size = 5 

for key, value in list(topology_data.items())[:print_size]:
    print(f"{key}: {value}")

16
12D3KooWH9rfYNKMkNncYJxS7BH41ThPZUYe3FNkbfmJAa4n5r3x: {'source_node_address': '0x5a5bf3d3ce59cd304f198b86c1a78adfadf31f83', 'channels_balance': 9202.300000000007}
12D3KooWL16nW1Z2dLvyZWzr9ZZwoLTeuSfaKSeX2BjucHwSoEwJ: {'source_node_address': '0xd30f8f6e5865d7ec947e101b1d6a183e9776ba40', 'channels_balance': 7755.100000000006}
12D3KooWGyY39vD8J2VGEDjTCD3eEyvV4YrnKM9NCQa6SYJKczrR: {'source_node_address': '0xcbe8726c80cc0d7751b9545dd5a4b5b0e53e383d', 'channels_balance': 7754.900000000006}
12D3KooWNYi2kG5cdeEUBvjemZRUkPVmAeXsSGVrX9QHnEiMfh8w: {'source_node_address': '0xa4642c066c1f8927db9d34abab599af784a2cff0', 'channels_balance': 9102.300000000005}
12D3KooWScC7bj5YdjDLDX3ibxQjkJLeHPRGk4fdNr7jWn9ugYko: {'source_node_address': '0xfbfc5497e511ebed91fb467fcb032046a5ad5e49', 'channels_balance': 0.24999999999999992}


In [14]:
subgraph_data = get_subgraph_data()
print(len(subgraph_data))

print_size = 5 

for key, value in list(subgraph_data.items())[:print_size]:
    print(f"{key}: {value}")


256
0xb55ba361c60ce0851f1f3451225b2f953ee70404: {'safe_address': '0x00133125ccdf4ea1231a47e073c616f358b2d5a8', 'wxHOPR_balance': '12258.755553838447062483'}
0x2168fcd793a3967fa4bdd66f534c4fc811124439: {'safe_address': '0x01f1d2f347ea987b5cf3ed383146feda5265f38a', 'wxHOPR_balance': '46412.387878358780883268'}
0xed04f9fbf9160793fff7532df3860c70862bff4e: {'safe_address': '0x0420bd44fe87a855a11c9fd42b3f42203b03dec9', 'wxHOPR_balance': '30000.007400087394399044'}
0xfcc30ccecf890362d66194659f4850acbe84b08b: {'safe_address': '0x042ddd9d9b99ed1a08eb5c5a3feae5e7a1732e82', 'wxHOPR_balance': '10000'}
0xf3e7672a909fd8c113fc5c53dda1f38f79d7a184: {'safe_address': '0x04b21235a04d7468bdd79de8a68341b7be0a71fa', 'wxHOPR_balance': '61208.146712387571348229'}


### Merge Subgraph and Topology Data 

In [21]:
def merge_topology_subgraph(topology_dict: dict, subgraph_dict: dict):
    """
    Merge metrics and subgraph data with the unique peer IDs, addresses,
    balance links.
    :param: topology_dict: A dict mapping peer IDs to node addresses.
    :param: subgraph_dict: A dict containing subgraph data with safe address as the key.
    :returns: A dict with peer ID as the key and the merged information.
    """
    merged_result = {}

    # Merge based on peer ID with the channel topology as the baseline
    for peer_id, data in topology_dict.items():
        seen_in_subgraph = False

        source_node_address = data["source_node_address"]
        if source_node_address in subgraph_dict:
            subgraph_data = subgraph_dict[source_node_address]
            data["safe_address"] = subgraph_data["safe_address"]
            data["safe_balance"] = float(subgraph_data["wxHOPR_balance"])
            data["total_balance"] = data["channels_balance"] + data["safe_balance"]

            seen_in_subgraph = True
            # print(f"Source node address for {peer_id} found in subgraph")

        if seen_in_subgraph:
            merged_result[peer_id] = data

    return merged_result

In [22]:
merged_data = merge_topology_subgraph(topology_data, subgraph_data)
print(len(merged_data))

print_size = 5 

for key, value in list(merged_data.items())[:print_size]:
    print(f"{key}: {value}")


16
12D3KooWH9rfYNKMkNncYJxS7BH41ThPZUYe3FNkbfmJAa4n5r3x: {'source_node_address': '0x5a5bf3d3ce59cd304f198b86c1a78adfadf31f83', 'channels_balance': 9202.300000000007, 'safe_address': '0xdf9be8bdb5ae4a130e861e5158c95667e7b2c0cb', 'safe_balance': 15817.7, 'total_balance': 25020.000000000007}
12D3KooWL16nW1Z2dLvyZWzr9ZZwoLTeuSfaKSeX2BjucHwSoEwJ: {'source_node_address': '0xd30f8f6e5865d7ec947e101b1d6a183e9776ba40', 'channels_balance': 7755.100000000006, 'safe_address': '0x4afa6a5265ae7ba332e886be3bce5b16c861dd9f', 'safe_balance': 17264.9, 'total_balance': 25020.000000000007}
12D3KooWGyY39vD8J2VGEDjTCD3eEyvV4YrnKM9NCQa6SYJKczrR: {'source_node_address': '0xcbe8726c80cc0d7751b9545dd5a4b5b0e53e383d', 'channels_balance': 7754.900000000006, 'safe_address': '0x5445a497292c8e669e7d3419be68de23c450c56f', 'safe_balance': 17265.1, 'total_balance': 25020.000000000004}
12D3KooWNYi2kG5cdeEUBvjemZRUkPVmAeXsSGVrX9QHnEiMfh8w: {'source_node_address': '0xa4642c066c1f8927db9d34abab599af784a2cff0', 'channels_ba

### TODO 

In [29]:
def stake_transformation(values, slope, curvature, threshold):
    transformed_stakes = []

    for x in values:
        if x <= 10e3:
            transformed_stakes.append(1e-20)
        elif x <= threshold:
            f_x = slope * x
            transformed_stakes.append(f_x)
        else:
            g_x = slope * threshold  + (x - threshold) ** (1 / curvature)
            transformed_stakes.append(g_x)

    return transformed_stakes

def compute_probabilities(stakes: list):
    sum_stakes = sum(stakes)
    return [s / sum_stakes for s in stakes]

def compute_rewards(probabilities: list, budget: int):
    return [p * budget for p in probabilities]

def compute_apy(opt: dict, data: list, percentage: bool = False, average: bool = True, on_stake: bool = False):
    transformed_stakes = stake_transformation(data, **opt["model_arguments"])

    probabilities = compute_probabilities(transformed_stakes)

    rewards = compute_rewards(probabilities, opt["budget"])

    if percentage:
        factor = 100
    else:
        factor = 1

    period_apy = [r / s for r, s in zip(rewards, data if on_stake else transformed_stakes)]
    yearly_apy = [apy * 12 / opt["period_in_months"] * factor for apy in period_apy]
    
    apy = sum(yearly_apy) / len(yearly_apy) if average else yearly_apy  
  
    return apy

def factor_and_prefix(value):
    factor = int(int(log10(value))/3)*3
    if factor < 3:
        prefix = ""
    elif factor < 6:
        prefix = "k"
    elif factor < 9:
        prefix = "M"

    return 10**factor, prefix

def probabilistic_apy(datas: list[list], options: list[dict], steps:int = 100):
    result_template = {"apys": [], "average": 0}

    results = [deepcopy(result_template) for _ in range(len(options))]

    for idx, opt in enumerate(options):
        for _ in range(steps):
            stakes = []
            for data, count in zip(datas, opt["data_count"]):
                if not count:
                    continue
                if count == "all":
                    stakes.extend(data)
                else:
                    stakes.extend(random.sample(data, count))

            tf_stakes = stake_transformation(stakes, **opt["model_arguments"])
            
            apy = compute_apy(opt, tf_stakes, percentage=False)

            results[idx]["apys"].append(apy)

        results[idx]["average"] = np.mean(results[idx]["apys"])
        results[idx]["std"] = np.std(results[idx]["apys"])

    return results

def generate_simulation_graph(datas:list[list], options: list[dict], steps: int = 200, title: str = None, cols:int=3):
    rows = int(len(options) / cols + 0.5)
    if rows * cols < len(options):
        rows += 1
        
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols*6,rows*6), dpi=300)

    axes = [axes] if isinstance(axes, Axes) else axes.flatten()


    results = probabilistic_apy(datas, options, steps)

    for opt, result, ax in zip(options, results, axes):
        max_stakes = max([max(data) for data, count in zip(datas, opt["data_count"]) if count])
        stakes = np.linspace(0, max_stakes, 1000)
        factor, prefix = factor_and_prefix(stakes[-1])
        
        tf_stakes = stake_transformation(stakes, **opt["model_arguments"])

        data_points = []
        for data, count in zip(datas, opt["data_count"]):
            if not count:
                continue
            if count == "all":
                data_points.extend(data)
            else:
                data_points.extend(random.sample(data, count))
       
        tf_data_points = stake_transformation(data_points, **opt["model_arguments"])

        stakes_for_plt = [s/factor for s in stakes]
        tf_stakes_for_plt = [s/factor for s in tf_stakes]

        data_points_for_plt = [s/factor for s in data_points]
        tf_data_points_for_plt = [s/factor for s in tf_data_points]
        
        messages_per_second = opt["budget"] / (opt["ticket_options"]["price"] * opt["ticket_options"]["winning_probability"]) / (opt["period_in_months"] * 30 * 24 * 60 * 60)


        ax.plot(stakes_for_plt, tf_stakes_for_plt, label=opt["legend"])
        ax.scatter(data_points_for_plt, tf_data_points_for_plt, s=12, alpha=0.8, c="#ff7f0e")

        ax.set_xlabel(f"Stake (/{prefix}/HOPR)")
        ax.set_ylabel(f"Transformed stake (/{prefix}/HOPR)")
        ax.text(0.1,
                0.8,
                f"APY: {result['average']:.2%} (+- {result['std']:.2%})",
                transform=ax.transAxes, 
                horizontalalignment='left', 
                bbox=dict(facecolor='red', alpha=0.5),)
        ax.text(0.1,
                0.7,
                f"Messages/s: {messages_per_second:.2f}",
                transform=ax.transAxes, 
                horizontalalignment='left', 
                bbox=dict(facecolor='blue', alpha=0.5),)
        
        # set the x and y scale equal
        ax.set_aspect('equal', adjustable='box')
        ax.set_ylim(ax.get_xlim())
        
        ax.legend()
        ax.grid()

    # remove unused axes
    for ax in axes[len(options):]:
        ax.remove()
        
    if title:
        if len(options) == 1:
            plt.title(title, fontsize=10)
        else:
            fig.suptitle(title, fontsize=20)
    plt.subplots_adjust(top=0.9)
    plt.show()

def generate_simulation_graph_simple(options: list[dict], steps: int, title: str = None):
    rows = 1
    cols = int(len(options) / rows + 0.5)
    if rows * cols < len(options):
        cols += 1
        
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols*6,rows*6), dpi=300)

    axes = [axes] if isinstance(axes, Axes) else axes.flatten()

    for opt, ax in zip(options, axes):        
        stakes = np.linspace(0, 1e6, steps)
        
        tf_stakes = stake_transformation(stakes, **opt["model_arguments"])
        
        ax.plot(stakes, tf_stakes)
        ax.set_xlabel(f"Stakes (HOPR)")
        ax.set_ylabel(f"Transformed Stakes (HOPR)")
    
        # set the x and y scale equal
        ax.set_aspect('equal', adjustable='box')
        ax.set_ylim(ax.get_xlim())
        
        ax.grid()

    # remove unused axes
    for ax in axes[len(options):]:
        ax.remove()
        
    if title:
        fig.suptitle(title, fontsize=20)
    plt.subplots_adjust(top=0.9)
    plt.show()

def generate_simulation_table(datas:list[list], options: list[dict], imposed_counts: list, steps: int = 200):
    
    apy_array = []
    messages_array = []
    combined_array = []

    for opt in options:
        temp_apy_array = []
        temp_messages_array = []
        temp_combined_array = []

        for count in imposed_counts:
            opt["data_count"] = count

            result = probabilistic_apy(datas, [opt], steps)[0]
            
            messages_per_second = opt["budget"] / (opt["ticket_options"]["price"] * opt["ticket_options"]["winning_probability"]) / (opt["period_in_months"] * 30 * 24 * 60 * 60)

            temp_apy_array.append(f"{result['average']:.2%}")
            temp_messages_array.append(f"{messages_per_second:.2f}")
            temp_combined_array.append(f"{result['average']:.2%} / {messages_per_second:.2f}m/s")

        apy_array.append(temp_apy_array)
        messages_array.append(temp_messages_array)
        combined_array.append(temp_combined_array)

    return apy_array, messages_array, combined_array

In [30]:
raw_all = pd.read_csv("all_01H7Z22K1VRTXCWJJJFV2A64VP.csv", low_memory=False)
raw_nft = pd.read_csv("nft_01H7Z2W99SMWJ01YN552WSRADV.csv", low_memory=False)

raw_data = pd.merge(raw_all, raw_nft, on="account", how="left")

condition = raw_data["token_id"].isnull()

staking_info_nft_holders = list(raw_data[~condition]["actual_stake_x"])
staking_info_non_nft_holders = list(raw_data[condition]["actual_stake_x"])
staking_info_all = list(raw_data["actual_stake_x"])

datas = [staking_info_nft_holders, staking_info_non_nft_holders, [500_000/0.05]]

# aim for 15% APY
prefered = {
    "data_count": ["all", 0, 0],
    "budget": 100_000, # fixed
    "period_in_months": 1, # fixed
    "model_arguments": {
        "slope": 1, # fixed
        "curvature":1.4,
        "threshold":75e3,
    },
    "ticket_options": { # fixed
        "price": 1, # fixed
        "winning_probability": 1, # fixed
    },
    "legend": "",
}


### Economic model

In [ ]:
prefered_vars = [deepcopy(prefered) for _ in range(5)]

prefered_vars[0]["data_count"] = [250, 0, 0]
prefered_vars[1]["data_count"] = [310, 0, 0]
prefered_vars[2]["data_count"] = [370, 0, 0]
prefered_vars[3]["data_count"] = ["all", 0, 0]
prefered_vars[4]["data_count"] = ["all", "all", 0]

for idx, opt in enumerate(prefered_vars[:-1]):
    opt["legend"] = f"(month {idx})"

prefered_vars[-1]["legend"] = f"(long-term)"

for idx, opt in enumerate(prefered_vars):
    opt["legend"] = f"{opt['data_count'][0]} NFT and {opt['data_count'][1]} non-NFT holders\n{opt['legend']}"

generate_simulation_graph(datas, prefered_vars, title=f"Realistic scenario: 250 node runners day 1 + 15/week\nbudget of {prefered_vars[0]['budget']/1000:.0f}k every {prefered_vars[0]['period_in_months']} month, ticket price at 0.001")

In [ ]:
apys = compute_apy(prefered, staking_info_nft_holders, percentage=True, average=False)
average = np.mean([apy for apy, stake in zip(apys, staking_info_nft_holders) if stake > 10e3])

print(f"Average APY: {average}")
print(f"Maximum APY: {max(apys)}")

plt.plot(staking_info_nft_holders, apys)
plt.axhline(average, color="red", linestyle="--")
plt.show()


In [ ]:
nft_holder_counts = [200, "all"]
options = [deepcopy(prefered) for _ in range(len(nft_holder_counts))]

for opt, count in zip(options, nft_holder_counts):
    opt["data_count"][0] = count
    opt["legend"] = f"{count} NFT holder"

options.append(deepcopy(prefered))

options[-1]["data_count"] = ["all", "all", 0]
options[-1]["legend"] = "NFT holders and non-NFT holders"


generate_simulation_graph(datas, options, title=f"Increasing network size (budget: {options[0]['budget']/1000:.0f}k, ticket price: {options[0]['ticket_options']['price']})", cols=3)

In [ ]:
slopes = [1, 0.75, 0.5]
options_nft = [deepcopy(prefered) for _ in range(len(slopes))]

for opt_nft, slope in zip( options_nft, slopes):
    opt_nft["data_count"] = ["all", 0, 0]
    opt_nft["model_arguments"]["slope"] = slope
    opt_nft["legend"] = f"slope of {slope} (all NFT holders)"

generate_simulation_graph(datas, options_nft, 200, title=f"Reducing slope (budget: {options[0]['budget']/1000:.0f}k, ticket price: {options[0]['ticket_options']['price']}, all NFT holders)")

In [ ]:
curvatures = [1, 1.125, 1.25]
options = [deepcopy(prefered) for _ in range(len(curvatures))]

for opt, curvature in zip(options, curvatures):
    opt["model_arguments"]["curvature"] = curvature
    opt["legend"] = f"curvature of {curvature}"

generate_simulation_graph(datas, options, 200, title=f"Increasing curvature (budget: {options[0]['budget']/1000:.0f}k, ticket price: {options[0]['ticket_options']['price']}, all NFT holders)")



In [ ]:
ticket_prices = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]
options = [deepcopy(prefered) for _ in range(len(ticket_prices))]

for opt, ticket_price in zip(options, ticket_prices):
    opt["ticket_options"]["price"] = ticket_price
    opt["legend"] = f"ticket price of {ticket_price}"

generate_simulation_graph(datas, options, title=f"Increasing ticket price (budget: {options[0]['budget']/1000:.0f}k, all NFT holders)")

In [ ]:
thresholds = [50e3, 100e3, 250e3]
options = [deepcopy(prefered) for _ in range(len(thresholds))]

for opt, threshold in zip(options, thresholds):
    opt["model_arguments"]["threshold"] = threshold
    opt["legend"] = f"threshold at {ticket_price}"


generate_simulation_graph(datas, options, title=f"Increasing threshold (budget: {options[0]['budget']/1000:.0f}k, ticket price: {options[0]['ticket_options']['price']}, all NFT holders)")

In [ ]:
options = [{
    "model_arguments": {
        "slope": 1,
        "curvature":1.125,
        "threshold":400e3,
    }
},
{
    "model_arguments": {
        "slope": 1,
        "curvature":1.125,
        "threshold":750e3,
    }
}
]

generate_simulation_graph_simple(options, 5000, title="whale-threshold (c) at 400k vs 750k")

In [ ]:
options = [{
    "model_arguments": {
        "slope": 1,
        "curvature":1.1,
        "threshold":400e3,
    }
},
{
    "model_arguments": {
        "slope": 1,
        "curvature":1.200,
        "threshold":400e3,
    }
},
{
    "model_arguments": {
        "slope": 1,
        "curvature":1.3,
        "threshold":400e3,
    }
}
]

generate_simulation_graph_simple(options, 5000, title="increasing curvature parameter (b)")

In [ ]:
print(f"min stake: {min(staking_info_nft_holders):.2f} HOPR")
print(f"max stake: {max(staking_info_nft_holders):.2f} HOPR")
print(f"average stake: {np.mean(staking_info_nft_holders):.2f} HOPR")
print(f"median stake: {np.median(staking_info_nft_holders):.2f} HOPR")
print(f"total reward: {sum(list(raw_data[~condition]['rewards_till_now'])):.2f} HOPR")


In [ ]:
ticket_prices = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]
options = [deepcopy(prefered) for _ in range(len(ticket_prices))]

for opt, ticket_price in zip(options, ticket_prices):
    opt["data_count"] = ["all", "all", 0]
    opt["ticket_options"]["price"] = ticket_price
    opt["legend"] = f"ticket price of {ticket_price}"

generate_simulation_graph(datas, options, title=f"Increasing ticket price (budget: {options[0]['budget']/1000:.0f}k, all NFT holders)")

### Table results

In [42]:
counts = [
    [250, 0, 0],
    [310, 0, 0],
    [370, 0, 0],
    ["all", 0, 0],
    ["all", "all", 0],
]
columns = [f"{count[0]} NFT and {count[1]} non-NFT" for count in counts]

In [ ]:
curvatures = [1, 1.125, 1.25, 1.375, 1.4, 1.5]
rows = [f"curvature: {curvature:.3f}" for curvature in curvatures]
options = [deepcopy(prefered) for _ in range(len(curvatures))]

for opt, curvature in zip(options, curvatures):
    opt["model_arguments"]["curvature"] = curvature

df_combined = pd.DataFrame(generate_simulation_table(datas, options, counts)[2], rows, columns)

display(df_combined)

In [ ]:
thresholds = [50e3, 75e3, 100e3, 150e3, 250e3]
rows = [f"threshold: {int(threshold/1e3)}k" for threshold in thresholds]
options = [deepcopy(prefered) for _ in range(len(thresholds))]

for opt, threshold in zip(options, thresholds):
    opt["model_arguments"]["threshold"] = threshold

df_combined = pd.DataFrame(generate_simulation_table(datas, options, counts)[2], rows, columns)

display(df_combined)

In [ ]:
ticket_prices = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]
rows = [f"ticket price: {price}" for price in ticket_prices]
options = [deepcopy(prefered) for _ in range(len(ticket_prices))]

for opt, price in zip(options, ticket_prices):
    opt["ticket_options"]["price"] = price

df_combined = pd.DataFrame(generate_simulation_table(datas, options, counts)[2], rows, columns)

display(df_combined)

In [ ]:

_, axes = plt.subplots(2, 3, figsize=(14, 8), dpi=300, sharey=True)
axes = axes.flatten()
options = deepcopy(prefered_vars)

for opt, ax in zip(options, axes):
    stakes = []

    for data, count in zip(datas, opt["data_count"]):
        if not count:
            continue
        if count == "all":
            stakes.extend(data)
        else:
            stakes.extend(random.sample(data, count))        

    stakes = sorted(stakes)

    lin_range = np.linspace(0, max(stakes), 1000)

    apys = compute_apy(opt, stakes, percentage=True, average=False, on_stake=True)
    global_apy = probabilistic_apy(datas, [opt], steps=200)[0]["average"] * 100

    rewards = [apy*stake/100 / 12 for apy, stake in zip(apys, stakes)]
    
    average_reward = np.mean([reward for reward, stake in zip(rewards, stakes) if stake > 10e3])
    median_reward = np.median([reward for reward, stake in zip(rewards, stakes) if stake > 10e3])

    factor, prefix = factor_and_prefix(stakes[-1])

    stakes = [s/factor for s in stakes]
    
    ax.plot(stakes, rewards, alpha=0.5)
    ax.scatter(stakes, rewards, s=8, c="#ff7f0e", alpha=0.5)
    ax.text(0.95,
            0.85,
            opt["legend"],
            transform=ax.transAxes,
            horizontalalignment='right')    
    ax.text(0.95,
            0.2,
            f"Median reward: {median_reward:.2f} HOPR",
            transform=ax.transAxes,
            horizontalalignment='right',
            bbox=dict(facecolor='orange', alpha=0.5),)
    ax.text(0.95,
            0.1,
            f"Average reward: {average_reward:,.2f} HOPR",
            transform=ax.transAxes,
            horizontalalignment='right',
            bbox=dict(facecolor='orange', alpha=0.5),)
    

    ax.set_xlabel(f"Stake (/{prefix}/HOPR)")
    ax.set_ylabel("Reward (HOPR)")

# remove unused subplots
for ax in axes[len(options):]:
    ax.remove()

plt.show()

In [ ]:

_, axes = plt.subplots(2, 3, figsize=(14, 8), dpi=300, sharey=True)
axes = axes.flatten()
options = deepcopy(prefered_vars)

for opt, ax in zip(options, axes):
    stakes = []

    for data, count in zip(datas, opt["data_count"]):
        if not count:
            continue
        if count == "all":
            stakes.extend(data)
        else:
            stakes.extend(random.sample(data, count))        

    stakes = sorted(stakes)

    lin_range = np.linspace(0, max(stakes), 1000)

    apys = compute_apy(opt, stakes, percentage=True, average=False, on_stake=True)
    global_apy = probabilistic_apy(datas, [opt], steps=200)[0]["average"] * 100
    
    average_apy = np.mean([apy for apy, stake in zip(apys, stakes) if stake > 10e3])
    median_apy = np.median([apy for apy, stake in zip(apys, stakes) if stake > 10e3])

    factor, prefix = factor_and_prefix(stakes[-1])

    stakes = [s/factor for s in stakes]
    
    ax.plot(stakes, apys, alpha=0.5)
    ax.scatter(stakes, apys, s=8, c="#ff7f0e", alpha=0.5)
    ax.text(0.95,
            0.85,
            opt["legend"],
            transform=ax.transAxes,
            horizontalalignment='right')    
    ax.text(0.95,
            0.75,
            f"Average APY: {average_apy:,.2f} %",
            transform=ax.transAxes,
            horizontalalignment='right',
            bbox=dict(facecolor='orange', alpha=0.5),)
    ax.text(0.95,
        0.65,
        f"Median APY: {median_apy:,.2f} %",
        transform=ax.transAxes,
        horizontalalignment='right',
        bbox=dict(facecolor='orange', alpha=0.5),)

    ax.set_xlabel(f"Stake (/{prefix}/HOPR)")
    ax.set_ylabel("APY (%)")

# remove unused subplots
for ax in axes[len(options):]:
    ax.remove()

plt.show()

In [ ]:
investor_node_count = [1, 25, 50, 100, 133, 400]
hopr_value = 0.05
investor_budget = 0.5e6 / hopr_value

def split_to_buckets(value, bucket_count, min_amount, max_amount):
    average = value / bucket_count

    if max_amount >= average and average >= min_amount:
        return [average] * bucket_count
    
    if average < min_amount:
        return split_to_buckets(value, bucket_count-1, min_amount, max_amount) + [0]
    
    filled_buckets = min(int(value / max_amount), bucket_count -1)
    remaining_value = value - filled_buckets * max_amount
    remaining_buckets = bucket_count - filled_buckets

    return [max_amount] * filled_buckets + split_to_buckets(remaining_value, remaining_buckets, min_amount, max_amount*2)

for node_count in investor_node_count:
    datas = [staking_info_nft_holders, staking_info_non_nft_holders, [investor_budget / node_count]*node_count]

    options = [deepcopy(prefered) for _ in range(2)]

    options[0]["data_count"] = ["all", 0, "all"]
    options[1]["data_count"] = ["all", "all", "all"]

    apys = probabilistic_apy(datas, options, 200)

    print(f"Investor with {int(investor_budget):,}HOPR and {node_count} nodes in the network")
    for idx, (opt, apy) in enumerate(zip(options, apys)):
        opt["legend"] = f"{opt['data_count'][0]} NFT and {opt['data_count'][1]} non-NFT holders {opt['data_count'][2]} investors nodes "

        if opt["data_count"][2] == "all":
            tf_stakes = stake_transformation(datas[2], **opt["model_arguments"])
        else:
            tf_stakes = [0]

        reward = [tf_stake * apy["average"] for tf_stake in tf_stakes] 

        print(f"    {opt['legend']}: {sum(reward):,.2f}HOPR -> {sum(reward)/investor_budget:.2%} APY (network APY: {apy['average']:.2%})")


    # generate_simulation_graph(datas, options, title=f"Realistic scenario: 250 node runners day 1 + 15/week\nbudget of {prefered_vars[0]['budget']/1000:.0f}k every {prefered_vars[0]['period_in_months']} month, ticket price at 0.001", cols=2)

In [ ]:
# datas = [staking_info_nft_holders, staking_info_non_nft_holders, [investor_budget / node_count]*node_count]

ct_candidates = [stake for stake in staking_info_all if stake > 10e3]
median_staker = ct_candidates[len(ct_candidates)//2]

median_stakes = [median_staker]

datas = [staking_info_nft_holders, staking_info_non_nft_holders, 0]

options = [deepcopy(prefered) for _ in range(2)]

options[0]["data_count"] = ["all", 0, 0]
options[1]["data_count"] = ["all", "all", 0]

apys = probabilistic_apy(datas, options, 200)

print(f"{median_staker:,}HOPR staker with {1} nodes in the network")
for idx, (opt, apy) in enumerate(zip(options, apys)):
    tf_stakes = stake_transformation(median_stakes, **opt["model_arguments"])

    reward = [tf_stake * apy["average"] for tf_stake in tf_stakes] 
    
    print(f"    : {sum(reward):,.2f}HOPR -> {sum(reward)/sum(median_stakes):.2%} APY (network APY: {apy['average']:.2%})")

In [ ]:
apy["average"] 

In [ ]:
len(staking_info_all)

In [ ]:
10e3